In [11]:
import numpy as np
import torch
import torchvision 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pickle
#import seaborn as sns; sns.set()
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
#from sklearn.utils.extmath import randomized_svd as rsvd
import numpy.linalg as LA
from moveSparseHyper_GPU import *


In [2]:
import torch

torch.cuda.is_available()
torch.cuda.current_device()
print(torch.cuda.device_count())
torch.cuda.get_device_name(1)

2


'NVIDIA GeForce RTX 3090 Ti'

In [3]:
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
traindataset = datasets.CIFAR10('data', train=True, download=True,transform=transform)
testdataset = datasets.CIFAR10('data', train=False, transform=transform)      

train_loader = torch.utils.data.DataLoader(traindataset, batch_size=60, shuffle=True, num_workers=0,drop_last=False)
#train_loader = cycle(train_loader)
test_loader = torch.utils.data.DataLoader(testdataset, batch_size=60, shuffle=False, num_workers=0,drop_last=True)



100%|██████████| 170498071/170498071 [00:05<00:00, 33768161.13it/s]


Extracting data/cifar-10-python.tar.gz to data


### Evaluation Script

In [5]:
# Function for Testing
def test(model, test_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #inputs, labels = inputs.to(device), labels.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

### Convolutional Network architecture(s)

In [13]:
import torch
import torch.nn as nn
import os

__all__ = [
    "ResNet",
    "resnet18",
    "resnet34",
    "resnet50",
]


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block,
        layers,
        num_classes=10,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
    ):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group

        # CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
        self.conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False
        )
        # END

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes,
                planes,
                stride,
                downsample,
                self.groups,
                self.base_width,
                previous_dilation,
                norm_layer,
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)

        return x


def _resnet(arch, block, layers, pretrained, progress, device, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        script_dir = 'cifar10_models'
        #script_dir = 'state_dicts/'
        
        state_dict = torch.load(script_dir + '/state_dicts/'+arch+'.pt', map_location=device)
        #state_dict = torch.load(script_dir + arch+'.pt', map_location=device)
        
#         script_dir = os.path.dirname(__file__)
#         state_dict = torch.load(
#             script_dir + "/state_dicts/" + arch + ".pt", map_location=device
#         )
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet18", BasicBlock, [2, 2, 2, 2], pretrained, progress, device, **kwargs
    )


def resnet34(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet34", BasicBlock, [3, 4, 6, 3], pretrained, progress, device, **kwargs
    )


def resnet50(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet50", Bottleneck, [3, 4, 6, 3], pretrained, progress, device, **kwargs
    )


In [14]:
my_model = resnet18()
my_model = resnet18(pretrained=True)

### Determine vector towards sparse hyperplane

In [9]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()

def calcWtsVec_fromNet(net):

    wtsVector_collection = np.array([])
    
    idx = 0;
    for n, p in net.named_parameters():
        
        wtsVec = p.view(-1,1);
        wtsVec = wtsVec.detach().cpu().numpy();
        wtsVector_collection = np.append( wtsVector_collection, wtsVec )
        
    return wtsVector_collection


def convertWtsVec_network(wtsVec):

    for numNetworks in range(1):
    
        net3 = copy.deepcopy(net_CIFAR)
        state_dict = net3.state_dict();

        ctr = 0;
        for n, p in state_dict.items():
            #print(p.shape)
            
            temp = p.view(-1,1);
            temp2 = list(p.size())
            #print(len(temp), n, len(temp2), ctr)
            
            if (len(temp) == 1 or '.running' in n):
                continue
            
            val = wtsVec[ctr: ctr + len(temp)];
            val = torch.Tensor(val).view(p.shape)
            
            if len(temp2) == 4:
                
                for idx in range(val.shape[0]):
                    p[idx,:,:,:] = val[idx,:,:,:]
                    
            elif len(temp2) == 2:
                
                for idx in range(val.shape[0]):
                    p[idx,:] = val[idx,:]
                    
            else:
                
                for idx in range(val.shape[0]):
                    p[idx] = val[idx];
                    
            ctr = ctr + len(temp)
            
    return net3
        


def findVec_sparseHyperplane(net, sparsity):
    
    wts_curr = calcWtsVec_fromNet(net)
    
    # Find the p% lowest mag weights of the network.
    wts_fin = calcWtsVec_fromNet(net)
    
    numWtsPrune = round(sparsity / 100 * len(wts_fin))
    
    # find min absolute wts and set the mink elements in mask to 0.
    idx = np.argsort(abs(wts_fin))
    
    wts_fin[idx[:numWtsPrune]] = 0.0
    vec_hyperplane = wts_fin - wts_curr
    vec_hyperplane = vec_hyperplane / LA.norm(vec_hyperplane, 2)
    
    return vec_hyperplane, wts_fin

### Move network towards the sparse hyperplane

In [12]:
def moveSparseNet_bhDist(net, sparsity, fname, num_epochs=2):

    ctr_run = 0
    
    print('inside function:', fname)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    net.train()
    bs = 200;
    
    train_loader = torch.utils.data.DataLoader(traindataset, batch_size=bs, shuffle=True, num_workers=0,drop_last=False)
    test_loader = torch.utils.data.DataLoader(testdataset, batch_size=bs, shuffle=False, num_workers=0,drop_last=True)


    
    f_softmax = nn.Softmax(dim=1)
    
    net_ori = copy.deepcopy(net)
    opt1 = NGD(net.parameters(), lr=0.01)
    #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    lambda1 = 0.1;
    accVec = []
    dist_hyper = []
    acc_sparse = []
    
    _, wts_sparse = findVec_sparseHyperplane(net, sparsity)
    net_sp = convertWtsVec_network(wts_sparse)
    acc_sp = test(net_sp, test_loader, criterion)
    wts_inter = calcWtsVec_fromNet(net)
    
    accVec.append(test(net, test_loader, criterion))
    dist_hyper.append(LA.norm(wts_sparse - wts_inter,2))
    acc_sparse.append(acc_sp)
    
    print("distHyper", dist_hyper[-1])    
    
    for epoch in range(num_epochs):
        for batch_idx, (imgs, targets) in enumerate(train_loader):

            opt1.zero_grad()
            #imgs, targets = next(train_loader)
            imgs, targets = imgs.to(device), targets.to(device)
            #output = net(imgs)

            output = f_softmax(net(imgs))
            output_ori = f_softmax(net_ori(imgs))

            output_ori = output_ori.detach()
            epsAdd = max(1e-12, torch.min(output_ori)*1e-3)
            loss = torch.sum(-torch.log(torch.sum(torch.sqrt(output*output_ori+epsAdd), axis=1)));

            #loss = criterion(output, targets)
            #loss.backward()

            [_, wts_sparse] = findVec_sparseHyperplane(net, sparsity)
            # =====================backward==================
            opt1.zero_grad()
            loss.backward()

            #plot_grad_flow(autoenc_ch.named_parameters)
            opt1.step(wts_sparse, lambda1)


            if batch_idx % 50 == 0:
                accVec.append(test(net, test_loader, criterion))
                #storeWts_manifold2(net, wtsDict)
                model2 = copy.deepcopy(net)
                wts_inter = calcWtsVec_fromNet(net)
                dist_hyper.append(LA.norm(wts_sparse-wts_inter,2))

                net_sp = convertWtsVec_network(wts_sparse)
                acc_sp = test(net_sp, test_loader, criterion)
                acc_sparse.append(acc_sp)
                print('Distance from projection in hyperplane', dist_hyper[-1])
                print('Accuracy', accVec[-1], acc_sp)
                pickle.dump([accVec, dist_hyper, net, acc_sparse], open(fname, "wb"))

                #pickle.dump([accVec, dist_hyper, net, acc_sp], open("ResNet18_"+str(sparsity)+"%sparse_BH_BS="+str(bs)+".p", "wb"))

                print("Loss val", loss)

                if dist_hyper[-1]<0.01:
                    break
    
    return (accVec, dist_hyper)

### Move ResNet-18 towards sparse hyperplane via FIP

In [15]:
net_CIFAR = copy.deepcopy(my_model)


sparsity = 95
num_epochs = 1000

[accVec, dist_hyper] = moveSparseNet_bhDist(net_CIFAR, sparsity, "ResNet18_"+str(sparsity)+"%sparse_bhMetric_batchsize=50_net1.p", num_epochs)

inside function: ResNet18_95%sparse_bhMetric_batchsize=50_net1.p
distHyper 1.7913771367847824
Distance from projection in hyperplane 1.7912635537772312
Accuracy 91.74 80.09
Loss val tensor(1.9261, device='cuda:0', grad_fn=<SumBackward0>)
Distance from projection in hyperplane 1.7909011148081582
Accuracy 90.99 78.22
Loss val tensor(2.1664, device='cuda:0', grad_fn=<SumBackward0>)
